# Practica 1

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import unicodedata
import re
import fuzzywuzzy
import nltk
from datetime import datetime

In [2]:
df=pd.read_csv("C:/Users/MX03932000/Documents/Personales/practica_1.csv")

C:\Users\MX03932000\Documents\BD SQL\WinPython\WPy64-3890\python-3.8.9.amd64\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [572]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774952 entries, 0 to 774951
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_id                   774952 non-null  int64  
 1   c_ao_hechos             774952 non-null  object 
 2   v_mes_hechos            774952 non-null  object 
 3   d_fecha_hechos          774952 non-null  object 
 4   t_delito                774952 non-null  object 
 5   v_categoria_delito      774952 non-null  object 
 6   v_fiscalia              774952 non-null  object 
 7   v_agencia               774952 non-null  object 
 8   v_unidad_investigacion  774952 non-null  object 
 9   v_colonia_hechos        774952 non-null  object 
 10  v_alcaldia_hechos       774952 non-null  object 
 11  d_fecha_inicio          774952 non-null  object 
 12  v_mes_inicio            774952 non-null  object 
 13  c_ao_inicio             774952 non-null  int64  
 14  t_calle_hechos      

# Parte I

In [4]:
df.columns

Index(['id', 'ao_hechos', 'mes_hechos', 'fecha_hechos', 'delito',
       'categoria_delito', 'fiscalia', 'agencia', 'unidad_investigacion',
       'colonia_hechos', 'alcaldia_hechos', 'fecha_inicio', 'mes_inicio',
       'ao_inicio', 'calle_hechos', 'calle_hechos2', 'longitud', 'latitud',
       'geopoint', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [6]:
#quitamos columnas excedentes
df=df[['id', 'ao_hechos', 'mes_hechos', 'fecha_hechos', 'delito',
       'categoria_delito', 'fiscalia', 'agencia', 'unidad_investigacion',
       'colonia_hechos', 'alcaldia_hechos', 'fecha_inicio', 'mes_inicio',
       'ao_inicio', 'calle_hechos', 'calle_hechos2', 'longitud', 'latitud',
       'geopoint']]

In [7]:
#Prefijos para los tipos de variables
#La siguiente nomenclatura se trabajara en el módulo
# "id" - "Identificadores"
# "c_"  - Variables numericas : Discretas y continuas
# "v_"  - Variables categoricas
# "d_"  - Variables tipo fecha
# "t_"  - Variables de texto : comentarios, descripciones, url

id_feats=['id']
c_feats=['ao_hechos','ao_inicio','longitud', 'latitud','geopoint']
v_feats=['mes_hechos','categoria_delito','fiscalia','agencia'
         ,'unidad_investigacion','colonia_hechos','alcaldia_hechos',
        'mes_inicio',]
d_feats=['fecha_hechos','fecha_inicio',]
t_feats=['delito','calle_hechos','calle_hechos2']

In [8]:
c_feats_new=["c_"+x for x in c_feats]
v_feats_new=["v_"+x for x in v_feats]
d_feats_new=["d_"+x for x in d_feats]
t_feats_new=["t_"+x for x in t_feats]
id_feats_new=["id_"+x for x in id_feats]

In [9]:
df.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True)
df.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
df.rename(columns=dict(zip(t_feats,t_feats_new)),inplace=True)
df.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)
df.rename(columns=dict(zip(id_feats,id_feats_new)),inplace=True)

# Duplicados

In [10]:
df.duplicated().sum()

0

In [11]:
df.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05 17:00:00,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01 16:40:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN IZTACALCO,IZC-2,UI-1SD,GABRIEL RAMOS MILLÃN SECCIÃN TLACOTAL,IZTACALCO,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30 20:00:00,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-8,UI-1CD,CONSEJO AGRARISTA MEXICANO,IZTAPALAPA,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16 16:00:00,ROBO A REPARTIDOR CON VIOLENCIA,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-9,UI-2SD,DESARROLLO URBANO QUETZALCOATL,IZTAPALAPA,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21 20:40:00,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-2SD,INFONAVIT IZTACALCO,IZTACALCO,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [12]:
df.duplicated(subset=["id_id"]).sum()

0

# Completitud

In [13]:
df.isnull().sum(0)

id_id                          0
c_ao_hechos                    0
v_mes_hechos                   0
d_fecha_hechos                 0
t_delito                       0
v_categoria_delito             0
v_fiscalia                     0
v_agencia                      0
v_unidad_investigacion         0
v_colonia_hechos               0
v_alcaldia_hechos              0
d_fecha_inicio                 0
v_mes_inicio                   0
c_ao_inicio                    0
t_calle_hechos                 0
t_calle_hechos2           461276
c_longitud                     1
c_latitud                      0
c_geopoint                     0
dtype: int64

In [14]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

In [15]:
completitud(df)

,columna,total,completitud
0,t_calle_hechos2,461276,40.476830
1,c_longitud,1,99.999871
2,id_id,0,100.000000
3,t_calle_hechos,0,100.000000
4,c_ao_inicio,0,100.000000
5,v_mes_inicio,0,100.000000
6,d_fecha_inicio,0,100.000000
7,v_alcaldia_hechos,0,100.000000
8,c_latitud,0,100.000000
9,v_colonia_hechos,0,100.000000


# Conformidad y orden

In [16]:
aux=df.copy()

In [17]:
#FECHAS
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774952 entries, 0 to 774951
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_id                   774952 non-null  int64  
 1   c_ao_hechos             774952 non-null  object 
 2   v_mes_hechos            774952 non-null  object 
 3   d_fecha_hechos          774952 non-null  object 
 4   t_delito                774952 non-null  object 
 5   v_categoria_delito      774952 non-null  object 
 6   v_fiscalia              774952 non-null  object 
 7   v_agencia               774952 non-null  object 
 8   v_unidad_investigacion  774952 non-null  object 
 9   v_colonia_hechos        774952 non-null  object 
 10  v_alcaldia_hechos       774952 non-null  object 
 11  d_fecha_inicio          774952 non-null  object 
 12  v_mes_inicio            774952 non-null  object 
 13  c_ao_inicio             774952 non-null  int64  
 14  t_calle_hechos      

In [82]:
#modificamos d_fecha_hechos 
aux["d_fecha_hechos"]=pd.to_datetime(aux["d_fecha_hechos"],format="%Y-%m-%d",errors="ignore")


In [19]:
#d_fecha_hechos con registros raros
raros_fh= aux[aux["d_fecha_hechos"].str.contains("[a-zA-Z]+",regex=True)]
raros_fh


,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
11833,12353,2016,Julio,martes| 12 de julio de 2016| 20:35:00 (),ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-2,UI-2CD,ESPARTACO,COYOACAN,2016-07-13 11:33:10,Julio,2016,CALZADA DE TLALPAN,A LA ALTURA DEL PUENTE DE DIVISION DEL NTE,-99.138068,19.319530,"19.3195304998,-99.1380683"
11834,12354,2019,Febrero,domingo| 10 de febrero de 2019| 3:00:00 (),FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMEN...,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-5,UI-2SD,JUÃREZ,CUAUHTEMOC,2019-02-22 20:32:00,Febrero,2019,HAMBURGO,NaN,-99.166148,19.425556,"19.4255561013,-99.1661481997"
11835,12355,2016,Mayo,jueves| 12 de mayo de 2016| 17:00:00 (),VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,ESCUADRÃN 201,IZTAPALAPA,2016-05-20 23:06:10,Mayo,2016,JAVIER MARTINEZ,NaN,-99.110802,19.363601,"19.3636009998,-99.110802"
11836,12356,2019,Febrero,miércoles| 20 de febrero de 2019| 8:30:00 (),DESPOJO,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-4,UI-2CD,MARTÃN CARRERA,GUSTAVO A MADERO,2019-02-22 21:05:00,Febrero,2019,MARTIN CARRERA,NaN,-99.107573,19.488494,"19.4884936991,-99.1075730999"
11837,12357,2016,Julio,miércoles| 13 de julio de 2016| 5:00:00 (),ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,SAN PEDRO,IZTAPALAPA,2016-07-13 12:42:04,Julio,2016,CJON. DEL 57,6O. CALLEJON DEL 57 (SIN REGISTRO DELS AP),-99.086085,19.364485,"19.36448547,-99.0860848837"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728000,759935,2016,Noviembre,viernes| 4 de noviembre de 2016| 14:00:00 (),ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN VENUSTIANO CARRANZA,VC-3,UI-2CD,MERCED BALBUENA,VENUSTIANO CARRANZA,2016-11-04 15:09:11,Noviembre,2016,ANILLO DE CIRCUNVALACION,NaN,-99.125946,19.422818,"19.4228182466,-99.1259457412"
728001,759936,2017,Septiembre,viernes| 1 de septiembre de 2017| 21:00:00 (),ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-7,UI-1SD,HIPÃDROMO CONDESA,CUAUHTEMOC,2017-09-02 20:56:16,Septiembre,2017,PUERTO REAL,CASI CON CAMPECHE,-99.176931,19.410389,"19.4103893859,-99.1769312192"
728002,759937,2016,Octubre,viernes| 28 de octubre de 2016| 21:00:00 (),ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN TLAHUAC,TLH-1,UI-2SD,LA TURBA,TLAHUAC,2016-11-04 15:38:16,Noviembre,2016,CISNE,NaN,-99.05918,19.296112,"19.2961116171,-99.0591802464"
728003,759938,2016,Noviembre,jueves| 3 de noviembre de 2016| 16:00:00 (),ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN AZCAPOTZALCO,AZ-1,UI-2SD,AMPLIACIÃN SAN PEDRO XALPA,AZCAPOTZALCO,2016-11-04 15:51:42,Noviembre,2016,GUSTAVO BAZAN,FRANCISCO VILLA,-99.215136,19.474942,"19.474942244,-99.2151362046"


In [20]:
#hacemos el tratamiento para transformarlos
raros_fh["d_fecha_hechos"]=raros_fh["d_fecha_hechos"].map(lambda x: x.split("|")[1].lstrip().replace("de","").replace(" ","/").replace("//","-"))

<ipython-input-20-7ea942e6f992>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raros_fh["d_fecha_hechos"]=raros_fh["d_fecha_hechos"].map(lambda x: x.split("|")[1].lstrip().replace("de","").replace(" ","/").replace("//","-"))


In [21]:
raros_fh.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
11833,12353,2016,Julio,12-julio-2016,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-2,UI-2CD,ESPARTACO,COYOACAN,2016-07-13 11:33:10,Julio,2016,CALZADA DE TLALPAN,A LA ALTURA DEL PUENTE DE DIVISION DEL NTE,-99.138068,19.319530,"19.3195304998,-99.1380683"
11834,12354,2019,Febrero,10-febrero-2019,FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMEN...,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-5,UI-2SD,JUÃREZ,CUAUHTEMOC,2019-02-22 20:32:00,Febrero,2019,HAMBURGO,NaN,-99.166148,19.425556,"19.4255561013,-99.1661481997"
11835,12355,2016,Mayo,12-mayo-2016,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,ESCUADRÃN 201,IZTAPALAPA,2016-05-20 23:06:10,Mayo,2016,JAVIER MARTINEZ,NaN,-99.110802,19.363601,"19.3636009998,-99.110802"
11836,12356,2019,Febrero,20-febrero-2019,DESPOJO,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-4,UI-2CD,MARTÃN CARRERA,GUSTAVO A MADERO,2019-02-22 21:05:00,Febrero,2019,MARTIN CARRERA,NaN,-99.107573,19.488494,"19.4884936991,-99.1075730999"
11837,12357,2016,Julio,13-julio-2016,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,SAN PEDRO,IZTAPALAPA,2016-07-13 12:42:04,Julio,2016,CJON. DEL 57,6O. CALLEJON DEL 57 (SIN REGISTRO DELS AP),-99.086085,19.364485,"19.36448547,-99.0860848837"


In [29]:
import time
import datetime
from time import strptime


In [50]:
#pruebas
datetime.datetime.strptime("5", "%m")

datetime.datetime(1900, 5, 1, 0, 0)

In [58]:
months_map = {'enero':1 , 'febrero':2,"marzo":3,"abril":4,"mayo":5,"junio":6,"julio":7,"agosto":8,"septiembre":9,"octubre":10,"noviembre":11,"diciembre":12}
def mapper(month):
    return months_map[month]

In [59]:
#pruebas de funcionamiento
mapper("febrero")

2

In [74]:
#funcion que asigna el numero de mes
def separa(letra):
    uno=letra.split("-")[0]
    months_map = {'enero':1 , 'febrero':2,"marzo":3,"abril":4,"mayo":5,"junio":6,"julio":7,"agosto":8,"septiembre":9,"octubre":10,"noviembre":11,"diciembre":12}
    dos=letra.split("-")[1]
    tres=letra.split("-")[2]
    final=uno+"-"+str(mapper(dos))+"-"+tres
    return final

In [73]:
#pruebas de funcionamiento
separa("13-julio-2016")

'13-7-2016'

In [75]:
#aplicamos la funcion previo a la transformación
raros_fh["d_fecha_hechos"]=raros_fh["d_fecha_hechos"].map(lambda x: separa(x))

<ipython-input-75-4c6f980110d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raros_fh["d_fecha_hechos"]=raros_fh["d_fecha_hechos"].map(lambda x: separa(x))


In [77]:
raros_fh.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
11833,12353,2016,Julio,12-7-2016,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-2,UI-2CD,ESPARTACO,COYOACAN,2016-07-13 11:33:10,Julio,2016,CALZADA DE TLALPAN,A LA ALTURA DEL PUENTE DE DIVISION DEL NTE,-99.138068,19.319530,"19.3195304998,-99.1380683"
11834,12354,2019,Febrero,10-2-2019,FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMEN...,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-5,UI-2SD,JUÃREZ,CUAUHTEMOC,2019-02-22 20:32:00,Febrero,2019,HAMBURGO,NaN,-99.166148,19.425556,"19.4255561013,-99.1661481997"
11835,12355,2016,Mayo,12-5-2016,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,ESCUADRÃN 201,IZTAPALAPA,2016-05-20 23:06:10,Mayo,2016,JAVIER MARTINEZ,NaN,-99.110802,19.363601,"19.3636009998,-99.110802"
11836,12356,2019,Febrero,20-2-2019,DESPOJO,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-4,UI-2CD,MARTÃN CARRERA,GUSTAVO A MADERO,2019-02-22 21:05:00,Febrero,2019,MARTIN CARRERA,NaN,-99.107573,19.488494,"19.4884936991,-99.1075730999"
11837,12357,2016,Julio,13-7-2016,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-4,UI-2SD,SAN PEDRO,IZTAPALAPA,2016-07-13 12:42:04,Julio,2016,CJON. DEL 57,6O. CALLEJON DEL 57 (SIN REGISTRO DELS AP),-99.086085,19.364485,"19.36448547,-99.0860848837"


In [78]:
#asignamos el tipo de dato
raros_fh["d_fecha_hechos"]=pd.to_datetime(raros_fh["d_fecha_hechos"],format="%d-%m-%Y")

<ipython-input-78-53f6520bcc3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raros_fh["d_fecha_hechos"]=pd.to_datetime(raros_fh["d_fecha_hechos"],format="%d-%m-%Y")


In [107]:
raros_fh.tail()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
728000,759935,2016,Noviembre,2016-11-04,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN VENUSTIANO CARRANZA,VC-3,UI-2CD,MERCED BALBUENA,VENUSTIANO CARRANZA,2016-11-04 15:09:11,Noviembre,2016,ANILLO DE CIRCUNVALACION,NaN,-99.125946,19.422818,"19.4228182466,-99.1259457412"
728001,759936,2017,Septiembre,2017-09-01,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-7,UI-1SD,HIPÃDROMO CONDESA,CUAUHTEMOC,2017-09-02 20:56:16,Septiembre,2017,PUERTO REAL,CASI CON CAMPECHE,-99.176931,19.410389,"19.4103893859,-99.1769312192"
728002,759937,2016,Octubre,2016-10-28,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN TLAHUAC,TLH-1,UI-2SD,LA TURBA,TLAHUAC,2016-11-04 15:38:16,Noviembre,2016,CISNE,NaN,-99.05918,19.296112,"19.2961116171,-99.0591802464"
728003,759938,2016,Noviembre,2016-11-03,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN AZCAPOTZALCO,AZ-1,UI-2SD,AMPLIACIÃN SAN PEDRO XALPA,AZCAPOTZALCO,2016-11-04 15:51:42,Noviembre,2016,GUSTAVO BAZAN,FRANCISCO VILLA,-99.215136,19.474942,"19.474942244,-99.2151362046"
728004,759939,2017,Septiembre,2017-09-03,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN XOCHIMILCO,XO-2,UI-1CD,LA NORIA III,XOCHIMILCO,2017-09-03 00:44:07,Septiembre,2017,QUINETZIN,NaN,-99.133468,19.270114,"19.2701142503,-99.1334684791"


In [130]:
#corroboramos que tenga el tipo de dato correcto
raros_fh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 11833 to 728004
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id_id                   310 non-null    int64         
 1   c_ao_hechos             310 non-null    object        
 2   v_mes_hechos            310 non-null    object        
 3   d_fecha_hechos          310 non-null    datetime64[ns]
 4   t_delito                310 non-null    object        
 5   v_categoria_delito      310 non-null    object        
 6   v_fiscalia              310 non-null    object        
 7   v_agencia               310 non-null    object        
 8   v_unidad_investigacion  310 non-null    object        
 9   v_colonia_hechos        310 non-null    object        
 10  v_alcaldia_hechos       310 non-null    object        
 11  d_fecha_inicio          310 non-null    object        
 12  v_mes_inicio            310 non-null    obj

In [144]:
aux2=aux.copy()
aux2.tail()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
774947,808866,2018,Julio,2018-07-09 22:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN MIGUEL HIDALGO,MH-1,UI-1SD,ANAHUAC,MIGUEL HIDALGO,2018-07-10 18:24:33,Julio,2018,LAGO ALBERTO,LAGO ITZEO,-99.182018,19.439418,"19.439418014,-99.1820184667"
774948,808867,2018,Julio,2018-07-29 05:00:00,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN MILPA ALTA,MIL-2,UI-1CD,SANTA CRUZ XOCHITEPEC,XOCHIMILCO,2018-07-29 12:37:27,Julio,2018,CALLEJON MORELOS,NaN,-99.134738,19.254935,"19.2549354998,-99.1347381"
774949,808868,2019,Febrero,2019-02-14 19:00:00,LESIONES INTENCIONALES POR ARMA DE FUEGO,LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO,INVESTIGACIÃN EN MIGUEL HIDALGO,H1,UI-1SD,INDUSTRIAL,GUSTAVO A MADERO,2019-02-15 12:50:00,Febrero,2019,CALZ. DE LOS MISTERIOS,NaN,-99.122943,19.473867,"19.473866796,-99.1229431996"
774950,808869,2019,Febrero,2019-02-15 10:00:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN AZCAPOTZALCO,AZ-4,UI-1SD,COSMOPOLITA,AZCAPOTZALCO,2019-02-15 13:04:00,Febrero,2019,CERDEÃA,NaN,-99.163384,19.473593,"19.4735930736,-99.163384042"
774951,808870,2017,Abril,2017-04-01 10:00:00,ABUSO DE CONFIANZA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN AGENCIAS DE ATENCIÃN ESPECI...,AER,UI-1SD,AEROPUERTO INTERNACIONAL BENITO JUÃREZ,VENUSTIANO CARRANZA,2017-05-26 17:04:07,Mayo,2017,CAP. AVIADOR CARLOS LEON,NaN,-99.083974,19.435153,"19.4351530369,-99.0839742938"


In [145]:
#modificamos para que pueda ser transformado
aux2["d_fecha_hechos"]=aux2["d_fecha_hechos"].map(lambda x: x.split(" ")[0])

In [125]:
raros_fh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 11833 to 728004
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id_id                   310 non-null    int64         
 1   c_ao_hechos             310 non-null    object        
 2   v_mes_hechos            310 non-null    object        
 3   d_fecha_hechos          310 non-null    datetime64[ns]
 4   t_delito                310 non-null    object        
 5   v_categoria_delito      310 non-null    object        
 6   v_fiscalia              310 non-null    object        
 7   v_agencia               310 non-null    object        
 8   v_unidad_investigacion  310 non-null    object        
 9   v_colonia_hechos        310 non-null    object        
 10  v_alcaldia_hechos       310 non-null    object        
 11  d_fecha_inicio          310 non-null    object        
 12  v_mes_inicio            310 non-null    obj

In [133]:
#ubicamos los id de los casos raros ya modificados previamente para hacer la transformación al complemento del df
idraros=raros_fh["id_id"]
idraros

11833      12353
11834      12354
11835      12355
11836      12356
11837      12357
           ...  
728000    759935
728001    759936
728002    759937
728003    759938
728004    759939
Name: id_id, Length: 310, dtype: int64

In [146]:
aux2.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN IZTACALCO,IZC-2,UI-1SD,GABRIEL RAMOS MILLÃN SECCIÃN TLACOTAL,IZTACALCO,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-8,UI-1CD,CONSEJO AGRARISTA MEXICANO,IZTAPALAPA,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,ROBO A REPARTIDOR CON VIOLENCIA,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-9,UI-2SD,DESARROLLO URBANO QUETZALCOATL,IZTAPALAPA,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-2SD,INFONAVIT IZTACALCO,IZTACALCO,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [147]:
#hacemos el filtro del df a solo los casos que no hemos transformado
fh_=aux2[np.logical_not(aux2.id_id.isin(idraros))]
fh_.count()

id_id                     774642
c_ao_hechos               774642
v_mes_hechos              774642
d_fecha_hechos            774642
t_delito                  774642
v_categoria_delito        774642
v_fiscalia                774642
v_agencia                 774642
v_unidad_investigacion    774642
v_colonia_hechos          774642
v_alcaldia_hechos         774642
d_fecha_inicio            774642
v_mes_inicio              774642
c_ao_inicio               774642
t_calle_hechos            774642
t_calle_hechos2           313542
c_longitud                774641
c_latitud                 774642
c_geopoint                774642
dtype: int64

In [ ]:
n.head()

In [148]:
#transformamos al formato correcto
fh_["d_fecha_hechos"]=pd.to_datetime(fh_["d_fecha_hechos"],format="%Y-%m-%d")

<ipython-input-148-c744ad48c8a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fh_["d_fecha_hechos"]=pd.to_datetime(fh_["d_fecha_hechos"],format="%Y-%m-%d")


In [152]:
#verificamos formato
fh_.head(1)


,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"


In [153]:
raros_fh.head(1)

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
11833,12353,2016,Julio,2016-07-12,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,INVESTIGACIÃN EN BENITO JUÃREZ,BJ-2,UI-2CD,ESPARTACO,COYOACAN,2016-07-13 11:33:10,Julio,2016,CALZADA DE TLALPAN,A LA ALTURA DEL PUENTE DE DIVISION DEL NTE,-99.138068,19.31953,"19.3195304998,-99.1380683"


In [154]:
#unimos los 2 df transformados para d_fecha_hechos
aux_fh=pd.concat([fh_,raros_fh])

In [157]:
aux_fh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 728004
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   v_mes_hechos            774952 non-null  object        
 3   d_fecha_hechos          774952 non-null  datetime64[ns]
 4   t_delito                774952 non-null  object        
 5   v_categoria_delito      774952 non-null  object        
 6   v_fiscalia              774952 non-null  object        
 7   v_agencia               774952 non-null  object        
 8   v_unidad_investigacion  774952 non-null  object        
 9   v_colonia_hechos        774952 non-null  object        
 10  v_alcaldia_hechos       774952 non-null  object        
 11  d_fecha_inicio          774952 non-null  object        
 12  v_mes_inicio            774952

In [201]:
limpieza_fh=aux_fh.copy()

In [203]:
limpieza_fh.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN IZTACALCO,IZC-2,UI-1SD,GABRIEL RAMOS MILLÃN SECCIÃN TLACOTAL,IZTACALCO,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-8,UI-1CD,CONSEJO AGRARISTA MEXICANO,IZTAPALAPA,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,ROBO A REPARTIDOR CON VIOLENCIA,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-9,UI-2SD,DESARROLLO URBANO QUETZALCOATL,IZTAPALAPA,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-2SD,INFONAVIT IZTACALCO,IZTACALCO,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [204]:
#cambio los valores que no tienen fechas, si no datos mezclados como estados.
limpieza_fh["d_fecha_hechos"]=pd.to_datetime(limpieza_fh["d_fecha_hechos"],format="%Y-%m-%d")

In [205]:
limpieza_fh["d_fecha_hechos"].head()

0   2016-01-05
1   2017-11-01
2   2015-12-30
3   2018-05-16
4   2018-05-21
Name: d_fecha_hechos, dtype: datetime64[ns]

In [216]:
limpieza_fi=limpieza_fh.copy()

In [ ]:
#hacemos el tratamiento para los 5 valores que tiene formato erroneo, usamos coerce para que los asigne a NaT

In [217]:
#hacemos el cambio de formato 
limpieza_fi["d_fecha_inicio"]=pd.to_datetime(limpieza_fi["d_fecha_inicio"],format="%Y-%m-%d",errors="coerce")

In [223]:
limpieza_fi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 728004
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   v_mes_hechos            774952 non-null  object        
 3   d_fecha_hechos          774952 non-null  datetime64[ns]
 4   t_delito                774952 non-null  object        
 5   v_categoria_delito      774952 non-null  object        
 6   v_fiscalia              774952 non-null  object        
 7   v_agencia               774952 non-null  object        
 8   v_unidad_investigacion  774952 non-null  object        
 9   v_colonia_hechos        774952 non-null  object        
 10  v_alcaldia_hechos       774952 non-null  object        
 11  d_fecha_inicio          774947 non-null  datetime64[ns]
 12  v_mes_inicio            774952

In [ ]:
#terminamos con el tratamiento de las variables fecha
#continuamos con el tratamiento para carácteres 

In [281]:
limpiezachar=limpieza_fi.copy()


# Limpieza de texto

In [282]:
limpiezachar.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN IZTACALCO,IZC-2,UI-1SD,GABRIEL RAMOS MILLÃN SECCIÃN TLACOTAL,IZTACALCO,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-8,UI-1CD,CONSEJO AGRARISTA MEXICANO,IZTAPALAPA,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,ROBO A REPARTIDOR CON VIOLENCIA,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-9,UI-2SD,DESARROLLO URBANO QUETZALCOATL,IZTAPALAPA,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-2SD,INFONAVIT IZTACALCO,IZTACALCO,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [283]:
char=["t_delito","v_categoria_delito","v_fiscalia","v_colonia_hechos","v_alcaldia_hechos","t_calle_hechos","t_calle_hechos2"]

In [284]:
#funcion que decodifica texto
def double_decode(x):
    return x.encode("utf-8").decode("utf-8").encode("latin-1").decode("utf-8")

In [285]:
#funcion para limpiar caracteres
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    text=str(text)
    #para lidiar con acentos
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    #para obtenerlo e UTF-8, la misma codificación que tiene python
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    #eliminar espacio antes y al final del string
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

In [286]:
#aplicamos nuestra funcion para hacer la decodificación
limpiezachar["t_delito"]=limpiezachar["t_delito"].map(lambda x: double_decode(x))

In [287]:
#notamos que tiene valores nulos a la categoría
limpiezachar["t_delito"].value_counts()

VIOLENCIA FAMILIAR                                 68004
ROBO DE OBJETOS                                    51510
ROBO A NEGOCIO SIN VIOLENCIA                       51172
FRAUDE                                             44198
DENUNCIA DE HECHOS                                 40377
                                                   ...  
ROBO A PASAJERO EN TREN SUBURBANO CON VIOLENCIA        1
2015-09-15 00:00:00                                    1
2016-06-14 16:30:00                                    1
PROCREACION ASISTIDA                                   1
PARRICIDIO                                             1
Name: t_delito, Length: 277, dtype: int64

In [288]:
#separamos los que tienen un formato diferente a la variable (con fecha)
limpieza_td=limpiezachar[limpiezachar["t_delito"].str.contains("([0-9]+)",regex=True)]

C:\Users\MX03932000\Documents\BD SQL\WinPython\WPy64-3890\python-3.8.9.amd64\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [291]:
#asignamos desconocido a los 4 registros distintos
limpieza_td["t_delito"]=limpieza_td["t_delito"].map(lambda x: "Desconocido")

<ipython-input-291-3cdd6ed67e0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  limpieza_td["t_delito"]=limpieza_td["t_delito"].map(lambda x: "Desconocido")


In [293]:
id_td=limpieza_td["id_id"]

505719    528001
505745    528028
505898    528188
506142    528446
Name: id_id, dtype: int64

In [295]:
#Guardamos en otro df los valores que no les hicimos tratamiento
limpios_td=limpiezachar[np.logical_not(limpiezachar.id_id.isin(id_td))]

In [297]:
concat_td=pd.concat([limpios_td,limpieza_td])

In [299]:
#aplicamos la funcion de limpieza de texto
concat_td["t_delito"]=concat_td["t_delito"].map(lambda x: clean_text(x) )


In [305]:
#terminamos de limpiar t_delitos
concat_td.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,lesiones culposas,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GRANJAS MODERNAS,GUSTAVO A MADERO,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,robo a transeunte en via publica con violencia,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,INVESTIGACIÃN EN IZTACALCO,IZC-2,UI-1SD,GABRIEL RAMOS MILLÃN SECCIÃN TLACOTAL,IZTACALCO,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,fraude,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN IZTAPALAPA,IZP-8,UI-1CD,CONSEJO AGRARISTA MEXICANO,IZTAPALAPA,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,robo a repartidor con violencia,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,INVESTIGACIÃN EN IZTAPALAPA,IZP-9,UI-2SD,DESARROLLO URBANO QUETZALCOATL,IZTAPALAPA,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,robo a transeunte de celular con violencia,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN CUAUHTEMOC,CUH-2,UI-2SD,INFONAVIT IZTACALCO,IZTACALCO,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [311]:
# limpiamos v_categoria_delito
limpieza_cd=concat_td.copy()

In [312]:
#notamos que no tiene otro tipo de dato en la categoría y solo haremos tratamiento de texto y decodificación.
limpieza_cd["v_categoria_delito"].value_counts()

DELITO DE BAJO IMPACTO                                     613398
HECHO NO DELICTIVO                                          47332
ROBO DE VEHÃCULO CON Y SIN VIOLENCIA                       41001
ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN VIOLENCIA      33120
ROBO A NEGOCIO CON VIOLENCIA                                11517
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        7178
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        5406
HOMICIDIO DOLOSO                                             3806
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                3038
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      2827
VIOLACIÃN                                                   1874
ROBO A CASA HABITACIÃN CON VIOLENCIA                        1830
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA      1403
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 724
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      408
SECUESTRO 

In [313]:
#decodificamos y limpiamos texto
limpieza_cd["v_categoria_delito"]=limpieza_cd["v_categoria_delito"].map(lambda x: clean_text(double_decode(x)))

In [315]:
limpieza_cd["v_categoria_delito"].value_counts()

delito de bajo impacto                                     613398
hecho no delictivo                                          47332
robo de vehiculo con y sin violencia                        41001
robo a transeunte en via publica con y sin violencia        33120
robo a negocio con violencia                                11517
robo a repartidor con y sin violencia                        7178
robo a pasajero a bordo del metro con y sin violencia        5406
homicidio doloso                                             3806
lesiones dolosas por disparo de arma de fuego                3038
robo a pasajero a bordo de microbus con y sin violencia      2827
violacion                                                    1874
robo a casa habitacion con violencia                         1830
robo a cuentahabiente saliendo del cajero con violencia      1403
robo a pasajero a bordo de taxi con violencia                 724
robo a transportista con y sin violencia                      408
secuestro 

In [317]:
#hacemos limpieza de v_fiscalia
limpieza_fis=limpieza_cd.copy()

In [319]:
limpieza_fis["v_fiscalia"].value_counts()

INVESTIGACIÃN EN CUAUHTEMOC                                                               99016
INVESTIGACIÃN EN IZTAPALAPA                                                               96023
INVESTIGACIÃN EN BENITO JUÃREZ                                                           64359
INVESTIGACIÃN EN GUSTAVO A. MADERO                                                        64353
INVESTIGACIÃN EN COYOACÃN                                                                45969
INVESTIGACIÃN EN MIGUEL HIDALGO                                                           44995
INVESTIGACIÃN EN ÃLVARO OBREGÃN                                                         44833
INVESTIGACIÃN EN TLALPAN                                                                  40322
INVESTIGACIÃN EN VENUSTIANO CARRANZA                                                      37677
INVESTIGACIÃN EN AZCAPOTZALCO                                                             34412
INVESTIGACIÃN EN IZTACALCO   

In [320]:
#decodificamos y limpiamos texto
limpieza_fis["v_fiscalia"]=limpieza_fis["v_fiscalia"].map(lambda x: clean_text(double_decode(x)))

In [323]:
limpieza_fis["v_fiscalia"].value_counts()

investigacion en cuauhtemoc                                                            99016
investigacion en iztapalapa                                                            96023
investigacion en benito juarez                                                         64359
investigacion en gustavo a madero                                                      64353
investigacion en coyoacan                                                              45969
investigacion en miguel hidalgo                                                        44995
investigacion en alvaro obregon                                                        44833
investigacion en tlalpan                                                               40322
investigacion en venustiano carranza                                                   37677
investigacion en azcapotzalco                                                          34412
investigacion en iztacalco                                            

In [325]:
#limpieza v_colonia_hechos
limpieza_ch=limpieza_fis.copy()

In [329]:
#decodificamos y limpiamos texto
limpieza_ch["v_colonia_hechos"]=limpieza_ch["v_colonia_hechos"].map(lambda x: clean_text(double_decode(x)))

In [335]:
limpieza_ch.tail()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
728004,759939,2017,Septiembre,2017-09-03,violencia familiar,delito de bajo impacto,investigacion en xochimilco,XO-2,UI-1CD,la noria iii,XOCHIMILCO,2017-09-03 00:44:07,Septiembre,2017,QUINETZIN,NaN,-99.133468,19.270114,"19.2701142503,-99.1334684791"
505719,528001,2017,Octubre,2017-10-09,desconocido,delito de bajo impacto,investigacion en gustavo a madero,GAM-4,UI-2CD,tepeyac insurgentes,GUSTAVO A MADERO,2017-10-09 11:03:30,Octubre,2017,RICARTE Y TICOMAN,NaN,-99.1211546404716,19.482295,"19.4822949828,-99.1211546405"
505745,528028,2015,Septiembre,2015-09-15,desconocido,delito de bajo impacto,investigacion en gustavo a madero,GAM-2,UI-3CD,barrio candelaria ticoman,GUSTAVO A MADERO,2016-05-12 14:21:49,Mayo,2016,PASO DEL NORTE,NaN,-99.1358799999999,19.521861,"19.5218609998,-99.13588"
505898,528188,2016,Febrero,2016-02-27,desconocido,delito de bajo impacto,investigacion en iztapalapa,IZP-6,UI-3SD,santa martha acatitla sur,IZTAPALAPA,2016-02-27 23:52:49,Febrero,2016,CALZADA IGNACIO ZARAGOZA,TITO FERRER,-99.012456,19.369853,"19.3698529998,-99.012456"
506142,528446,2016,Junio,2016-06-14,desconocido,delito de bajo impacto,investigacion en tlalpan,TLP-2,UI-1CD,barrio la lonja,TLALPAN,2016-06-15 09:59:34,Junio,2016,IZTACCIHUALTL,NaN,-99.1800321631259,19.290686,"19.290686271,-99.1800321631"


In [336]:
#limpieza v_alcaldia_hechos
limpieza_ahe=limpieza_ch.copy()

In [337]:
#decodificamos y limpiamos texto
limpieza_ahe["v_alcaldia_hechos"]=limpieza_ahe["v_alcaldia_hechos"].map(lambda x: clean_text(double_decode(x)))

In [338]:
limpieza_ahe["v_alcaldia_hechos"].value_counts()

cuauhtemoc                126168
iztapalapa                114682
gustavo a madero           76935
benito juarez              69342
coyoacan                   52767
miguel hidalgo             52502
alvaro obregon             51135
tlalpan                    44888
venustiano carranza        44474
azcapotzalco               38876
iztacalco                  33886
xochimilco                 24019
tlahuac                    17237
la magdalena contreras     11864
cuajimalpa de morelos      11205
milpa alta                  4972
Name: v_alcaldia_hechos, dtype: int64

In [340]:
limpieza_ahe.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,Enero,2016,SAN JUAN DE ARAGON,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,Noviembre,2017,SUR 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,Enero,2016,CUARTA CERRADA DE FRESNO,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,Mayo,2018,TARASCO,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,Mayo,2018,FRANCISCO DEL PASO Y TRONCOSO,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [341]:
#limpieza de t_calle_hechos y t_calle_hechos2
limpieza_calles=limpieza_ahe.copy()

In [342]:
#decodificamos
limpieza_calles["t_calle_hechos"]=limpieza_calles["t_calle_hechos"].map(lambda x: clean_text(double_decode(x)))

In [346]:
#decido no darle importancia al tratamiento de t_calle_hechos2 ya que tiene solo 40% de completitud, 
#si se requiere para alguna diferenciación la retomaré. 

In [347]:
limpieza_calles.head()

,id_id,c_ao_hechos,v_mes_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,v_mes_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,Enero,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,Enero,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,Noviembre,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,Noviembre,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,Diciembre,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,Enero,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,Mayo,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,Mayo,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,Mayo,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,Mayo,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [349]:
dflimpio=limpieza_calles.copy()

In [350]:
for col in dflimpio:
    print(col)
    display(dflimpio[col].value_counts())
    print("\n")

id_id


0         1
594551    1
645742    1
643695    1
600688    1
         ..
679393    1
673250    1
675299    1
685540    1
2047      1
Name: id_id, Length: 774952, dtype: int64



c_ao_hechos


2018    220464
2017    195552
2016    168535
2019    102565
2017     20221
         ...  
1973         1
2023         1
1977         1
1978         1
1998         1
Name: c_ao_hechos, Length: 109, dtype: int64



v_mes_hechos


...

-99.0701573999999    425
-99.1131053999999    311
-99.1968276999998    300
-99.0529625999999    236
-99.1705252999999    233
                    ... 
-99.2352888263691      1
-99.1502407771376      1
-99.0570185887788      1
-99.1277736308074      1
-99.1689669999999      1
Name: c_longitud, Length: 614877, dtype: int64



c_latitud


19.393029    441
19.307951    309
19.298169    250
19.263624    246
19.382933    241
            ... 
19.333687      1
19.270069      1
19.353952      1
19.433878      1
19.525335      1
Name: c_latitud, Length: 612315, dtype: int64



c_geopoint


19.3930293998,-99.0701574       441
19.3079511998,-99.1968277       309
19.2981688998,-99.1131054       250
19.2636241998,-99.1705253       246
19.3829333998,-99.0529626       241
                               ... 
19.4010495847,-99.1376493905      1
19.4704419998,-99.155062          1
19.2480816731,-99.1784006624      1
19.3517354688,-99.0074820972      1
19.4934335998,-99.1221731         1
Name: c_geopoint, Length: 613168, dtype: int64

In [ ]:
#v_mes_hechos y v_mes_inicio son variables que omitiremos, ya que son redundantes dadas las variables de fechas. 

In [351]:
dflimpio.columns

Index(['id_id', 'c_ao_hechos', 'v_mes_hechos', 'd_fecha_hechos', 't_delito',
       'v_categoria_delito', 'v_fiscalia', 'v_agencia',
       'v_unidad_investigacion', 'v_colonia_hechos', 'v_alcaldia_hechos',
       'd_fecha_inicio', 'v_mes_inicio', 'c_ao_inicio', 't_calle_hechos',
       't_calle_hechos2', 'c_longitud', 'c_latitud', 'c_geopoint'],
      dtype='object')

In [352]:
dflimpio=dflimpio[['id_id', 'c_ao_hechos', 'd_fecha_hechos', 't_delito',
       'v_categoria_delito', 'v_fiscalia', 'v_agencia',
       'v_unidad_investigacion', 'v_colonia_hechos', 'v_alcaldia_hechos',
       'd_fecha_inicio', 'c_ao_inicio', 't_calle_hechos',
       't_calle_hechos2', 'c_longitud', 'c_latitud', 'c_geopoint']]

In [353]:
dflimpio.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [573]:
limpieza_lista=dflimpio.copy()

In [574]:
limpieza_lista.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 506142
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   d_fecha_hechos          774952 non-null  datetime64[ns]
 3   t_delito                774952 non-null  object        
 4   v_categoria_delito      774952 non-null  object        
 5   v_fiscalia              774952 non-null  object        
 6   v_agencia               774952 non-null  object        
 7   v_unidad_investigacion  774952 non-null  object        
 8   v_colonia_hechos        774952 non-null  object        
 9   v_alcaldia_hechos       774952 non-null  object        
 10  d_fecha_inicio          774947 non-null  datetime64[ns]
 11  c_ao_inicio             774952 non-null  int64         
 12  t_calle_hechos          774952

In [673]:
cat_delito=limpieza_lista.copy()

In [675]:
mod_coordenadas=cat_delito.copy()

In [676]:
#modificar latitud y longitud
#Los valores posibles para la latitude son :
#-90,90

#Los valores posibles para la longitud son :
#-180,180



In [677]:
#haremos la limpieza para coordenadas
mod_coordenadas.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [678]:
mod_coordenadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 506142
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   d_fecha_hechos          774952 non-null  datetime64[ns]
 3   t_delito                774952 non-null  object        
 4   v_categoria_delito      774952 non-null  object        
 5   v_fiscalia              774952 non-null  object        
 6   v_agencia               774952 non-null  object        
 7   v_unidad_investigacion  774952 non-null  object        
 8   v_colonia_hechos        774952 non-null  object        
 9   v_alcaldia_hechos       774952 non-null  object        
 10  d_fecha_inicio          774947 non-null  datetime64[ns]
 11  c_ao_inicio             774952 non-null  int64         
 12  t_calle_hechos          774952

In [679]:
#cambio formato
mod_coordenadas["c_longitud"]=pd.to_numeric(mod_coordenadas["c_longitud"],errors='coerce')

In [680]:
mod_coordenadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 506142
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   d_fecha_hechos          774952 non-null  datetime64[ns]
 3   t_delito                774952 non-null  object        
 4   v_categoria_delito      774952 non-null  object        
 5   v_fiscalia              774952 non-null  object        
 6   v_agencia               774952 non-null  object        
 7   v_unidad_investigacion  774952 non-null  object        
 8   v_colonia_hechos        774952 non-null  object        
 9   v_alcaldia_hechos       774952 non-null  object        
 10  d_fecha_inicio          774947 non-null  datetime64[ns]
 11  c_ao_inicio             774952 non-null  int64         
 12  t_calle_hechos          774952

In [681]:
#separamos las logitudes que salen del rango
modificarlong=mod_coordenadas[mod_coordenadas["c_longitud"]>180]

In [682]:
#a los errores les doy "nan"
modificarlong["c_longitud"]=modificarlong["c_longitud"].map(lambda x: float('nan'))

<ipython-input-682-e603ec00470d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modificarlong["c_longitud"]=modificarlong["c_longitud"].map(lambda x: float('nan'))


In [683]:
modificarlong.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
506111,528414,2016,2016-06-05,robo de vehiculo de servicio particular sin vi...,robo de vehiculo con y sin violencia,investigacion en iztapalapa,IZP-7,UI-3CD,san simon culhuacan,iztapalapa,2016-06-05 20:16:48,2016,moctezuma,CARLOS DAVID ANDERSON,NaN,19.334764,"19.3347637527,-99.1055968715"
506112,528415,2016,2016-11-10,violencia familiar,delito de bajo impacto,investigacion en xochimilco,XO-2,UI-1SD,el jazmin,xochimilco,2016-11-16 11:33:52,2016,gardenia,HERMENEGILDO GALEANA,NaN,19.244908,"19.2449079064,-99.0887010805"
506113,528416,2016,2016-06-05,tentativa de robo,delito de bajo impacto,investigacion en gustavo a madero,GAM-5,UI-3CD,nueva atzacoalco,gustavo a madero,2016-06-05 21:36:22,2016,calle 314,CALLE 317,NaN,0.000000,"19.5007203909,-99.0844165732"
506114,528417,2016,2016-09-04,delitos electorales,delito de bajo impacto,investigacion para la atencion de asuntos espe...,B,UI-3CD,centro,cuauhtemoc,2017-01-04 12:08:25,2017,guatemala,CORREO MAYOR,NaN,19.434610,"19.4346103943,-99.129761151"
506116,528419,2016,2016-11-16,robo de accesorios de auto,delito de bajo impacto,investigacion en miguel hidalgo,MH-1,UI-1SD,popotla,miguel hidalgo,2016-11-16 13:20:45,2016,instituto de higiene,MAR ROJO,NaN,19.453671,"19.4536714272,-99.1801328369"


In [684]:
#guardo los id´s para poder diferenciarlos de mi df original y así poder filtrar
long_modificados_id=modificarlong['id_id']

In [685]:
#guardo mi df que no se hamodificado, para diferenciarlo
limpias_sinmodificados_lon=mod_coordenadas[np.logical_not(mod_coordenadas.id_id.isin(long_modificados_id))]


In [686]:
limpias_sinmodificados_lon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774943 entries, 0 to 506142
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774943 non-null  int64         
 1   c_ao_hechos             774943 non-null  object        
 2   d_fecha_hechos          774943 non-null  datetime64[ns]
 3   t_delito                774943 non-null  object        
 4   v_categoria_delito      774943 non-null  object        
 5   v_fiscalia              774943 non-null  object        
 6   v_agencia               774943 non-null  object        
 7   v_unidad_investigacion  774943 non-null  object        
 8   v_colonia_hechos        774943 non-null  object        
 9   v_alcaldia_hechos       774943 non-null  object        
 10  d_fecha_inicio          774938 non-null  datetime64[ns]
 11  c_ao_inicio             774943 non-null  int64         
 12  t_calle_hechos          774943

In [687]:
#uno mi df que acabo de limpiar con el que ya estaba limpio
limpiototal_long=pd.concat([limpias_sinmodificados_lon,modificarlong])

In [688]:
limpiototal_long.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774952 entries, 0 to 506120
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   774952 non-null  int64         
 1   c_ao_hechos             774952 non-null  object        
 2   d_fecha_hechos          774952 non-null  datetime64[ns]
 3   t_delito                774952 non-null  object        
 4   v_categoria_delito      774952 non-null  object        
 5   v_fiscalia              774952 non-null  object        
 6   v_agencia               774952 non-null  object        
 7   v_unidad_investigacion  774952 non-null  object        
 8   v_colonia_hechos        774952 non-null  object        
 9   v_alcaldia_hechos       774952 non-null  object        
 10  d_fecha_inicio          774947 non-null  datetime64[ns]
 11  c_ao_inicio             774952 non-null  int64         
 12  t_calle_hechos          774952

In [689]:
limpiototal_long.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"19.4810759998,-99.103527"
1,2,2017,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"19.3953976727,-99.1060723829"
2,3,2015,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"19.3224579998,-99.055887"
3,4,2018,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"19.3191405147,-99.0427862526"
4,5,2018,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"19.3865939284,-99.1115626682"


In [690]:
#ahora a limpiar latitud
limpiar_lat=limpiototal_long.copy()

In [691]:
limpiar_lat["c_latitud"].head()

0    19.481076
1    19.395398
2    19.322458
3    19.319141
4    19.386594
Name: c_latitud, dtype: float64

In [692]:
#filtro los que salgan del rango adecuado
sucios_lat=limpiar_lat[limpiar_lat["c_latitud"]>90]

In [693]:
#asigno nan a errores
sucios_lat["c_latitud"]=sucios_lat["c_latitud"].map(lambda x: float('nan'))

<ipython-input-693-d58d74d777aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucios_lat["c_latitud"]=sucios_lat["c_latitud"].map(lambda x: float('nan'))


In [694]:
#guardo los id´s para poder diferenciarlos de mi df original y así poder filtrar
lat_exsucio=sucios_lat["id_id"]

In [695]:
#filtro mi df sin errores
lat_limpio_ss=limpiar_lat[np.logical_not(limpiar_lat.id_id.isin(lat_exsucio))]

In [696]:
#uno mi df que acabo de limpiar con el que no tenía errores
lat_totallimpio=pd.concat([lat_limpio_ss,sucios_lat])

In [697]:
df_xgeo=lat_totallimpio.copy()

In [698]:
#defino una funcion que me limpie de manera provisional las coordenadas
def treatgeo(valor):
    lat=float(valor.split(",")[0])
    long=float(valor.split(",")[1])
    if (-90<lat and lat<90):
        lat
    else:
        lat="ninguno"
    if (-180<long and long<180):
        long
    else:
        long="ninguno"
        
    return(lat,long)
        

In [699]:
#aplico mi función
df_xgeo['c_geopoint']=df_xgeo['c_geopoint'].map(lambda x: treatgeo(x))

In [700]:
df_xgeo.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,d_fecha_inicio,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint
0,0,2016,2016-01-05,lesiones culposas,delito de bajo impacto,investigacion en gustavo a madero,GAM-6,UI-1CD,granjas modernas,gustavo a madero,2016-01-05 18:35:37,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"(19.4810759998, -99.103527)"
1,2,2017,2017-11-01,robo a transeunte en via publica con violencia,robo a transeunte en via publica con y sin vio...,investigacion en iztacalco,IZC-2,UI-1SD,gabriel ramos millan seccion tlacotal,iztacalco,2017-11-01 16:56:04,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"(19.3953976727, -99.1060723829)"
2,3,2015,2015-12-30,fraude,delito de bajo impacto,investigacion en iztapalapa,IZP-8,UI-1CD,consejo agrarista mexicano,iztapalapa,2016-01-05 21:12:42,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"(19.3224579998, -99.055887)"
3,4,2018,2018-05-16,robo a repartidor con violencia,robo a repartidor con y sin violencia,investigacion en iztapalapa,IZP-9,UI-2SD,desarrollo urbano quetzalcoatl,iztapalapa,2018-05-22 10:44:45,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"(19.3191405147, -99.0427862526)"
4,5,2018,2018-05-21,robo a transeunte de celular con violencia,delito de bajo impacto,investigacion en cuauhtemoc,CUH-2,UI-2SD,infonavit iztacalco,iztacalco,2018-05-22 10:52:18,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"(19.3865939284, -99.1115626682)"


In [702]:
df_listo=df_xgeo.copy()

# Parte II

In [716]:
#reducir tabla a sucesos de 2013 en adelante
dffiltroaño=df_listo[df_listo["d_fecha_hechos"] >='2013-01-01 00:00:00']

In [717]:
dffiltroaño["d_fecha_hechos"].min()

Timestamp('2013-01-01 00:00:00')

In [718]:
dffiltroaño.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 770614 entries, 0 to 506007
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   770614 non-null  int64         
 1   c_ao_hechos             770614 non-null  object        
 2   d_fecha_hechos          770614 non-null  datetime64[ns]
 3   t_delito                770614 non-null  object        
 4   v_categoria_delito      770614 non-null  object        
 5   v_fiscalia              770614 non-null  object        
 6   v_agencia               770614 non-null  object        
 7   v_unidad_investigacion  770614 non-null  object        
 8   v_colonia_hechos        770614 non-null  object        
 9   v_alcaldia_hechos       770614 non-null  object        
 10  d_fecha_inicio          770609 non-null  datetime64[ns]
 11  c_ao_inicio             770614 non-null  int64         
 12  t_calle_hechos          770614

In [707]:
#categorizar la variable categoria_delito'

In [719]:
cat_delito=dffiltroaño.copy()

In [720]:
cat_delito["v_categoria_delito"].value_counts()

delito de bajo impacto                                     609863
hecho no delictivo                                          46645
robo de vehiculo con y sin violencia                        40977
robo a transeunte en via publica con y sin violencia        33091
robo a negocio con violencia                                11509
robo a repartidor con y sin violencia                        7178
robo a pasajero a bordo del metro con y sin violencia        5402
homicidio doloso                                             3806
lesiones dolosas por disparo de arma de fuego                3032
robo a pasajero a bordo de microbus con y sin violencia      2827
violacion                                                    1835
robo a casa habitacion con violencia                         1826
robo a cuentahabiente saliendo del cajero con violencia      1402
robo a pasajero a bordo de taxi con violencia                 723
robo a transportista con y sin violencia                      408
secuestro 

In [721]:
#hago mis categorías para normalizar
#robos en general
robos=["robo de vehiculo con y sin violencia","robo a transeunte en via publica con y sin violencia","robo a negocio con violencia"
       ,"robo a repartidor con y sin violencia","robo a pasajero a bordo del metro con y sin violencia","robo a pasajero a bordo de microbus con y sin violencia"
      ,"robo a casa habitacion con violencia","robo a cuentahabiente saliendo del cajero con violencia","robo a pasajero a bordo de taxi con violencia"
      ,"robo a transportista con y sin violencia"]

In [722]:
#delitos de baja gravedad
delitos_grav_bajo=["delito de bajo impacto","hecho no delictivo"]

In [723]:
#homicidios
homicidios=["homicidio doloso"]

In [724]:
#lesiones por arma
armas=["lesiones dolosas por disparo de arma de fuego"]

In [725]:
#violaciones
violacion=["violacion"]

In [726]:
#secuestro
secuestro=["secuestro"]

In [765]:
#defino función para normalizar delitos
def norm_cat_delitos(n):
    if (n in robos):
        n="robos"
    if (n in delitos_grav_bajo):
        n="delitos_grav_bajo"
    if (n in homicidios):
        n="homicidios"
    if (n in armas):
        n="armas"
    if (n in violacion):
        n="violacion"
    elif(n in secuestro):
        n="secuestro"
    return (str(n))
    
    

In [766]:
#prueba de funcion
sec="hecho no delictivo"

In [768]:
type(norm_cat_delitos(sec))

str

In [769]:
#aplico mi funcion para normalizar cat. delitos
cat_delito["v_categoria_delito"]=cat_delito["v_categoria_delito"].map(lambda x: norm_cat_delitos(x) )

In [770]:
cat_delito["v_categoria_delito"].value_counts()

delitos_grav_bajo    656508
robos                105343
homicidios             3806
armas                  3032
violacion              1835
secuestro                90
Name: v_categoria_delito, dtype: int64

In [773]:
cat_delito["v_categoria_delito"]=cat_delito["v_categoria_delito"].astype(str)

In [ ]:
#normalizar la delegación según el mapa

In [793]:
norm_deleg=cat_delito.copy()

In [794]:
norm_deleg.columns

Index(['id_id', 'c_ao_hechos', 'd_fecha_hechos', 't_delito',
       'v_categoria_delito', 'v_fiscalia', 'v_agencia',
       'v_unidad_investigacion', 'v_colonia_hechos', 'v_alcaldia_hechos',
       'd_fecha_inicio', 'c_ao_inicio', 't_calle_hechos', 't_calle_hechos2',
       'c_longitud', 'c_latitud', 'c_geopoint'],
      dtype='object')

In [795]:
norm_deleg[ 'v_alcaldia_hechos'].value_counts()

cuauhtemoc                124713
iztapalapa                114257
gustavo a madero           76613
benito juarez              68920
coyoacan                   52559
miguel hidalgo             52235
alvaro obregon             50936
tlalpan                    44669
venustiano carranza        44280
azcapotzalco               38691
iztacalco                  33759
xochimilco                 23902
tlahuac                    17168
la magdalena contreras     11806
cuajimalpa de morelos      11150
milpa alta                  4956
Name: v_alcaldia_hechos, dtype: int64

In [796]:
#asigno mis variables de normalización
z_cen_pon=["cuauhtemoc","miguel hidalgo","alvaro obregon","azcapotzalco","cuajimalpa de morelos"]

In [797]:
z_sur=["benito juarez","coyoacan","tlalpan","la magdalena contreras"]

In [798]:
z_norte=["gustavo a madero","venustiano carranza","iztacalco"]

In [799]:
z_oriente=["iztapalapa","xochimilco","tlahuac","milpa alta"]

In [800]:
#defino mi funcion para normalizar delegaciones
def fun_norm_deleg(n):
    if (n in z_cen_pon):
        n="z_cen_pon"
    if (n in z_sur):
        n="z_sur"
    if (n in z_norte):
        n="z_norte"
    elif (n in z_oriente):
        n="z_oriente"
    
    return (str(n))
    

In [802]:
#aplico mi función para normalizar
norm_deleg['v_alcaldia_hechos']=norm_deleg['v_alcaldia_hechos'].map(lambda x:fun_norm_deleg(x))

In [804]:
norm_deleg['v_alcaldia_hechos'].value_counts()

z_cen_pon    277725
z_sur        177954
z_oriente    160283
z_norte      154652
Name: v_alcaldia_hechos, dtype: int64

In [805]:
norm_deleg.columns

Index(['id_id', 'c_ao_hechos', 'd_fecha_hechos', 't_delito',
       'v_categoria_delito', 'v_fiscalia', 'v_agencia',
       'v_unidad_investigacion', 'v_colonia_hechos', 'v_alcaldia_hechos',
       'd_fecha_inicio', 'c_ao_inicio', 't_calle_hechos', 't_calle_hechos2',
       'c_longitud', 'c_latitud', 'c_geopoint'],
      dtype='object')

In [806]:
#normalizamos la colonia

In [917]:
norm_col=norm_deleg.copy()

In [918]:
norm_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 770614 entries, 0 to 506007
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_id                   770614 non-null  int64         
 1   c_ao_hechos             770614 non-null  object        
 2   d_fecha_hechos          770614 non-null  datetime64[ns]
 3   t_delito                770614 non-null  object        
 4   v_categoria_delito      770614 non-null  object        
 5   v_fiscalia              770614 non-null  object        
 6   v_agencia               770614 non-null  object        
 7   v_unidad_investigacion  770614 non-null  object        
 8   v_colonia_hechos        770614 non-null  object        
 9   v_alcaldia_hechos       770614 non-null  object        
 10  d_fecha_inicio          770609 non-null  datetime64[ns]
 11  c_ao_inicio             770614 non-null  int64         
 12  t_calle_hechos          770614

In [919]:
norm_col["v_colonia_hechos"].value_counts()

centro                                 28294
doctores                               15956
del valle centro                       12525
roma norte                             10326
narvarte                                9262
                                       ...  
belen de las flores seccion turbina        1
campo de tiro los gamitos                  1
loma bonita                                1
iztacalco                                  1
san juan tepenahuac                        1
Name: v_colonia_hechos, Length: 1620, dtype: int64

In [920]:
# Reglas para normalización
#(0-9,431)--> col_indice_bajo
#(9,432,18,862)-->col_indice_medio
#(18,863,28294)-->col_indice_alto

In [922]:
#hacemos las apariciones un diccionario
ser=norm_col["v_colonia_hechos"].value_counts().to_dict()
ser

{'centro': 28294,
 'doctores': 15956,
 'del valle centro': 12525,
 'roma norte': 10326,
 'narvarte': 9262,
 'buenavista': 7793,
 'juarez': 7106,
 'polanco': 7071,
 'guerrero': 6747,
 'morelos': 6725,
 'agricola oriental': 6612,
 'santa maria la ribera': 5730,
 'agricola pantitlan': 5449,
 'central de abasto': 5217,
 'portales sur': 4562,
 'jardin balbuena': 4444,
 'pedregal de santo domingo': 4420,
 'obrera': 4354,
 'cuauhtemoc': 4043,
 'santa cruz atoyac': 3858,
 'hipodromo': 3787,
 'anahuac': 3780,
 'roma sur': 3773,
 'napoles': 3756,
 'lindavista': 3727,
 'granjas mexico': 3585,
 'el rosario': 3533,
 'moctezuma 2a seccion': 3530,
 'san pedro de los pinos': 3308,
 'guadalupe tepeyac': 3199,
 'tacuba': 3160,
 'lomas de chapultepec i seccion': 3128,
 'zona centro': 3040,
 'alamos': 3032,
 'lomas de san lorenzo': 3029,
 'san andres totoltepec': 2982,
 'desarrollo urbano quetzalcoatl': 2979,
 'del carmen': 2941,
 'tacubaya': 2881,
 'san rafael': 2821,
 'santa cruz meyehualco': 2801,
 'sa

In [929]:
#funcion para normalización
def norm_col_nuevo(n):
    if (ser[n] in range (0,9431)):
        m="col_indice_bajo"
    if (ser[n] in range (9431,18862)):
        m="col_indice_medio"
    elif (ser[n]in range (18862,28295)):
        m="col_indice_alto"
        
    return(m)

In [930]:
norm_col_nuevo('moctezuma 2a seccion')

'col_indice_bajo'

In [931]:
#aplico mi función de normalizacion
norm_col["v_colonia_hechos"]=norm_col["v_colonia_hechos"].map(lambda x: norm_col_nuevo(x))

In [932]:
norm_col["v_colonia_hechos"].value_counts()

col_indice_bajo     703513
col_indice_medio     38807
col_indice_alto      28294
Name: v_colonia_hechos, dtype: int64

In [ ]:
#normalizar delito,u.investigación y fiscalia
#paso1: se generará una nueva normalización para las 3 variables

In [933]:
norm_d_ui_f=norm_col.copy()

In [936]:
norm_d_ui_f.columns

Index(['id_id', 'c_ao_hechos', 'd_fecha_hechos', 't_delito',
       'v_categoria_delito', 'v_fiscalia', 'v_agencia',
       'v_unidad_investigacion', 'v_colonia_hechos', 'v_alcaldia_hechos',
       'd_fecha_inicio', 'c_ao_inicio', 't_calle_hechos', 't_calle_hechos2',
       'c_longitud', 'c_latitud', 'c_geopoint'],
      dtype='object')

In [941]:
#normalizaremos delitos segun su frecuencia 
delitos_=norm_d_ui_f['t_delito'].value_counts().to_dict()

In [942]:
#se haran las sig categorías:
#d_frec_alta--> rango de frecuencia(67890,11314]
#d_frec_moderada--> rango de frecuencia(11314,1214]
#d_frec_baja--> rango de frecuencia(1214,100]
#d_frec_inusual-->rango de frecuencia(100,0)
delitos_

{'violencia familiar': 67890,
 'robo de objetos': 51442,
 'robo a negocio sin violencia': 51153,
 'fraude': 42915,
 'denuncia de hechos': 39694,
 'amenazas': 36891,
 'robo a transeunte en via publica con violencia': 28860,
 'robo a transeunte de celular con violencia': 25753,
 'robo de accesorios de auto': 25315,
 'robo de objetos del interior de un vehiculo': 23633,
 'robo a casa habitacion sin violencia': 21930,
 'falsificacion de titulos al portador y documentos de credito publico': 21545,
 'robo de vehiculo de servicio particular sin violencia': 19443,
 'robo a transeunte de celular sin violencia': 18174,
 'abuso de confianza': 12668,
 'dano en propiedad ajena culposa por transito vehicular a automovil': 12484,
 'despojo': 12194,
 'robo a negocio con violencia': 11314,
 'narcomenudeo posesion simple': 9790,
 'dano en propiedad ajena intencional': 9117,
 'lesiones culposas por transito vehicular en colision': 9114,
 'lesiones intencionales por golpes': 8754,
 'robo a pasajero conduc

In [967]:
#se haran las sig categorías, segun la siguiente lógica:
#d_frec_alta--> rango de frecuencia(67890,11314]
#d_frec_moderada--> rango de frecuencia(11314,1214]
#d_frec_baja--> rango de frecuencia(1214,100]
#d_frec_inusual-->rango de frecuencia(100,0)

def delitos_frec_c(c):
    if (delitos_[c] in range (0,100)):
        o="d_frec_inusual"
    if (delitos_[c] in range (100,1214)):
        o="d_frec_baja"
    if (delitos_[c] in range (1214,11314)):
        o="d_frec_moderada"
    elif (delitos_[c] in range (11314,67891)):
        o="d_frec_alta"
        
    return(o)
    
    

In [968]:
#prueba de función
delitos_['robo a transportista y vehiculo pesado con violencia']

352

In [969]:
delitos_frec_c('robo a transportista y vehiculo pesado con violencia')


'd_frec_baja'

In [970]:
#aplico mi función
norm_d_ui_f["v_frec_delito"]=norm_d_ui_f['t_delito'].apply(lambda x: delitos_frec_c(x))

In [971]:
norm_d_ui_f["v_frec_delito"].value_counts()

d_frec_alta        523298
d_frec_moderada    212089
d_frec_baja         32122
d_frec_inusual       3105
Name: v_frec_delito, dtype: int64

In [972]:
#normalizamos u.investigación

In [ ]:
norm_d_ui_f

In [975]:
dic_ui=norm_d_ui_f['v_unidad_investigacion'].value_counts().to_dict()
dic_ui
#reglas de normalización
#(213021,94130]->alta_frec_ui
#(94130,722]->media_frec_ui
#(722,279)->baja_frec_ui
#(279,0)->inusual_frec_ui

{'UI-1SD': 213020,
 'UI-2SD': 118658,
 'UI-1CD': 108547,
 'UI-3SD': 97930,
 'UI-3CD': 96526,
 'UI-2CD': 94129,
 '3 CON DETENIDO 3 C/D': 3556,
 '2 CON DETENIDO 2 C/D': 3188,
 'UI-4SD': 3093,
 '1 CON DETENIDO 1 C/D': 2830,
 '2 S/D': 2808,
 '1 S/D': 2460,
 'FDS-6-02': 1280,
 '3 S/D': 1246,
 '4 SIN DETENIDO 4 S/D': 1245,
 'FDS-6-01': 1147,
 'FDS-5-01': 1003,
 'FDS-6-03': 987,
 'FDS-1-03': 768,
 'FDS-5-03': 756,
 'FDS-5-02': 722,
 'UI-7SD': 689,
 'FDS-2-01': 654,
 'FDS-2-02': 641,
 'FDS-1-02': 639,
 'FDS-4-03': 635,
 'UI-5SD': 633,
 'FDS-2-03': 617,
 '2 SIN DETENIDO': 616,
 '3 SIN DETENIDO': 596,
 'FDS-4-01': 588,
 'FDS-4-02': 586,
 'FDS-1-01': 560,
 '1 SIN DETENIDO': 538,
 'UI-6SD': 531,
 '3 SIN DETENIDO 3 S/D': 521,
 'FDS-3-02': 455,
 'FDS-3-01': 436,
 'FDS-3-03': 433,
 '1 SIN DETENIDO 1 S/D': 403,
 'B-2 SIN DETENIDO': 358,
 '2 SIN DETENIDO 2 S/D': 357,
 'D-6 SIN DETENIDO': 352,
 'B-4 SIN DETENIDO': 313,
 'B-3 SIN DETENIDO': 302,
 'Unidad de investigaciÃ³n 7 S/D': 286,
 'B-5 SIN DETENIDO'

In [976]:
#(213021,94130]->alta_frec_ui
#(94130,722]->media_frec_ui
#(722,279)->baja_frec_ui
#(279,0)->inusual_frec_ui
def delitos_frec_c(c):
    if (dic_ui[c] in range (94130,213021)):
        o="alta_frec_ui"
    if (dic_ui[c] in range (722,94130)):
        o="media_frec_ui"
    if (dic_ui[c] in range (279,722)):
        o="baja_frec_ui"
    elif (dic_ui[c] in range (0,279)):
        o="inusual_frec_ui"
        
    return(o)

In [978]:
norm_d_ui_f["v_frec_ui"]=norm_d_ui_f['v_unidad_investigacion'].map(lambda x: delitos_frec_c(x) )

In [979]:
#normalizamos fiscalia

In [984]:
dict_fiscal=norm_d_ui_f["v_fiscalia"].value_counts().to_dict()
dict_fiscal

{'investigacion en cuauhtemoc': 98223,
 'investigacion en iztapalapa': 95813,
 'investigacion en gustavo a madero': 64166,
 'investigacion en benito juarez': 64099,
 'investigacion en coyoacan': 45820,
 'investigacion en miguel hidalgo': 44845,
 'investigacion en alvaro obregon': 44730,
 'investigacion en tlalpan': 40176,
 'investigacion en venustiano carranza': 37556,
 'investigacion en azcapotzalco': 34310,
 'investigacion en iztacalco': 29497,
 'investigacion para la atencion de ninos ninas y adolescentes': 22533,
 'investigacion en xochimilco': 20492,
 'juzgados familiares': 16405,
 'investigacion en tlahuac': 15212,
 'investigacion en agencias de atencion especializadas': 14395,
 'investigacion para la atencion de delitos sexuales': 12912,
 'investigacion de los delitos cometidos por servidores publicos': 12493,
 'investigacion en magdalena contreras': 10187,
 'investigacion en cuajimalpa': 9533,
 'investigacion para la atencion del delito de robo de vehiculos y transporte': 6517,

In [986]:
def fisc_frecuencias(c):
    if (dict_fiscal[c] in range (37556,98224)):
        o="alta_frec_fisc"
    if (dict_fiscal[c] in range (6517,37556)):
        o="media_frec_fisc"
    elif (dict_fiscal[c] in range (0,6517)):
        o="baja_frec_fisc"
        
    return(o)

In [987]:
norm_d_ui_f["v_frec_fisc"]=norm_d_ui_f["v_fiscalia"].map(lambda x: fisc_frecuencias(x) )

In [988]:
norm_d_ui_f["v_frec_fisc"].value_counts()

alta_frec_fisc     535428
media_frec_fisc    204486
baja_frec_fisc      30700
Name: v_frec_fisc, dtype: int64

In [989]:
#hacemos el analisis de las 3 varibles para checar como podemos crear una nueva

In [990]:
norma_d_u_f=norm_d_ui_f.copy()

In [1007]:
nuevas_fechas=norma_d_u_f.copy()

In [ ]:
#cuatrimestres

In [1036]:
def cuatri(x):
    import math
    n=math.ceil(x.month/4.)
    return(n)

In [1034]:

nuevas_fechas['d_fecha_hechos_cuatrimestre']=nuevas_fechas['d_fecha_hechos'].map(lambda x: cuatri(x))

In [1035]:
nuevas_fechas['d_fecha_hechos_cuatrimestre'].value_counts()

1    283631
2    256997
3    229986
Name: d_fecha_hechos_cuatrimestre, dtype: int64

In [1037]:
nuevas_fechas.head()

,id_id,c_ao_hechos,d_fecha_hechos,t_delito,v_categoria_delito,v_fiscalia,v_agencia,v_unidad_investigacion,v_colonia_hechos,v_alcaldia_hechos,...,c_ao_inicio,t_calle_hechos,t_calle_hechos2,c_longitud,c_latitud,c_geopoint,v_frec_delito,v_frec_ui,v_frec_fisc,d_fecha_hechos_cuatrimestre
0,0,2016,2016-01-05,lesiones culposas,delitos_grav_bajo,investigacion en gustavo a madero,GAM-6,UI-1CD,col_indice_bajo,z_norte,...,2016,san juan de aragon,CALLE ANZAR,-99.103527,19.481076,"(19.4810759998, -99.103527)",d_frec_moderada,alta_frec_ui,alta_frec_fisc,1
1,2,2017,2017-11-01,robo a transeunte en via publica con violencia,robos,investigacion en iztacalco,IZC-2,UI-1SD,col_indice_bajo,z_norte,...,2017,sur 117,ORIENTE 100A,-99.106072,19.395398,"(19.3953976727, -99.1060723829)",d_frec_alta,alta_frec_ui,media_frec_fisc,3
2,3,2015,2015-12-30,fraude,delitos_grav_bajo,investigacion en iztapalapa,IZP-8,UI-1CD,col_indice_bajo,z_oriente,...,2016,cuarta cerrada de fresno,ENCINO (SIN REGISTRO DEL SAP),-99.055887,19.322458,"(19.3224579998, -99.055887)",d_frec_alta,alta_frec_ui,alta_frec_fisc,3
3,4,2018,2018-05-16,robo a repartidor con violencia,robos,investigacion en iztapalapa,IZP-9,UI-2SD,col_indice_bajo,z_oriente,...,2018,tarasco,ZAPOTECO,-99.042786,19.319141,"(19.3191405147, -99.0427862526)",d_frec_moderada,alta_frec_ui,alta_frec_fisc,2
4,5,2018,2018-05-21,robo a transeunte de celular con violencia,delitos_grav_bajo,investigacion en cuauhtemoc,CUH-2,UI-2SD,col_indice_bajo,z_norte,...,2018,francisco del paso y troncoso,ISLA,-99.111563,19.386594,"(19.3865939284, -99.1115626682)",d_frec_alta,alta_frec_ui,alta_frec_fisc,2


In [1039]:
#día de la semana
nuevas_fechas["d_fecha_hechos_wday"]=nuevas_fechas['d_fecha_hechos'].map(lambda x: x.isoweekday())

In [1040]:
nuevas_fechas["d_fecha_hechos_wday"].value_counts()

5    120704
3    116095
1    115295
2    114995
4    114510
6    100071
7     88944
Name: d_fecha_hechos_wday, dtype: int64

In [1045]:
#funcion para fin de semana
#1<-si es fin de semana
#0 <-si no es fin de semana

def finsem(s):
    
    if(s==6):
        m=1
    if(s==7):
        m=1
    else:
        m=0
    return(m)


In [1046]:
nuevas_fechas["d_fecha_hechos_weekend"]=nuevas_fechas["d_fecha_hechos_wday"].map(lambda x: finsem(x))

In [1047]:
nuevas_fechas["d_fecha_hechos_weekend"].value_counts()

0    681670
1     88944
Name: d_fecha_hechos_weekend, dtype: int64

In [1048]:
#día del evento:
nuevas_fechas["d_fecha_hechos_dia"]=nuevas_fechas['d_fecha_hechos'].map(lambda x: x.day)

In [1049]:
nuevas_fechas["d_fecha_hechos_dia"].value_counts()

1     29161
15    28565
5     25978
20    25817
4     25697
10    25647
7     25604
8     25580
22    25544
3     25384
16    25356
12    25270
23    25252
17    25242
14    25217
18    25202
13    25015
27    24980
11    24977
26    24912
19    24907
28    24773
2     24765
6     24736
9     24601
25    24472
21    24411
24    24076
30    22938
29    22506
31    14029
Name: d_fecha_hechos_dia, dtype: int64

# Parte III

In [1050]:
#creación del TAD

Paso 1:Divido mi train and test, 

Paso 2: Haría variables dummy con one hot encoder para la normalización que expuse en mi ejercicio para la categoría de delitos( k-1), así represento mis delitos de predicción.

Paso 3: Haría variables con count encoding para cada ventana de tiempo (hora, minuto y segundo) del día para representar los sucesos.

Paso 4:Haría un desfase de 1 semana para poder analizar las ocurrencias.
 